In [3]:
# 금통위 크롤링용 입니다. 
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta

import requests
from bs4 import BeautifulSoup
#import html2text
from datetime import datetime



def get_minutes_list(from_date='20050101'): # 기존 20050101 에서 20000101변경 
    prefix_addr = "https://www.bok.or.kr"
    from_date = datetime.strptime(from_date, '%Y%m%d')
    temp =0 # 몇번하는지 단순 순서 체크용입니다. 
    for pageIndex in range(1, 49): ## 46 페이지 == 2000,1,13, 49페이지 까지 있다. 전체 대상 
        url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(pageIndex)
        user_agent = 'Mozilla/5.0'
        headers = {'User-Agent': user_agent}
        page = requests.get(url, headers=headers)

        try: # 예외 처리용 혹시 접속 오류 등의 사항을 위해 
            soup = BeautifulSoup(page.content, 'html.parser')
            brdList = soup.find_all('span', class_='col m10 s10 x9 ctBx')
            datainfo = soup.find_all('div', class_='col s12 dataInfo')
    
            for post, data in zip(brdList, datainfo):
                guid = prefix_addr + post.a['href'][:-12] # -12 주의 머지?
                #print("이동중입니다. : ",guid)
                temp+=1
                dl_file(guid,temp)

        except:
            print("get url.content error and pass page{} it".format(pageIndex))


#다운로드용 
def dl_file(url,temp) :
    #print("url : ",url)
    user_agent = 'Mozilla/5.0'
    headers = {'User-Agent': user_agent}
    page = requests.get(url, headers=headers)

    #print("page.content : ",page.content)
    soup = BeautifulSoup(page.content, 'html.parser')
    file1 = soup.find('div', class_='addfile').find('ul').findAll('li')[1].find('a').attrs['href'] # 접속후 태그 가지고 오기 
    name = soup.find('h3', class_='subject').text # 이름 저장용입니다. 
    #subject
    #print("temp : ",file1," name : ",name)

    link1 = 'http://www.bok.or.kr'+file1

    file_res = requests.get(link1)
    
    #pdf
    with open('/content/text/{}.pdf'.format(name),'wb') as f : # 텍스트 폴더를 만들어야 합니다! 
      f.write(file_res.content)
      print("index : ",temp, " name : ",name," 파일 크롤링 ")

get_minutes_list(from_date='20050101')

print(" 파일 크롤링 py가 실행 완료 되었습니당! ~.~")




index :  1  name :  금융통화위원회 의사록(2022년도 제7차)(2022.4.14)   파일 크롤링 
index :  2  name :  금융통화위원회 의사록(2022년도 제6차)(2022.3.24)   파일 크롤링 
index :  3  name :  금융통화위원회 의사록(2022년도 제5차)(2022.3.10.)   파일 크롤링 
index :  4  name :  금융통화위원회 의사록(2022년도 제4차)(2022.2.24.)   파일 크롤링 
index :  5  name :  금융통화위원회 의사록(2022년도 제1차)(2022.1.14.)   파일 크롤링 
index :  6  name :  금융통화위원회 의사록(2021년도 제25차)(2021.12.23.)   파일 크롤링 
index :  7  name :  금융통화위원회 의사록(2021년도 제24차)(2021.12.9.)   파일 크롤링 
index :  8  name :  금융통화위원회 의사록(2021년도 제23차)(2021.11.25.)   파일 크롤링 
index :  9  name :  금융통화위원회 의사록(2021년도 제20차)(2021.10.12.)   파일 크롤링 
index :  10  name :  금융통화위원회 의사록(2021년도 제19차)(2021.9.24.)   파일 크롤링 
index :  11  name :  금융통화위원회 의사록(2021년도 제18차)(2021.9.9.)   파일 크롤링 
index :  12  name :  금융통화위원회 의사록(2021년도 제17차)(2021.8.26.)   파일 크롤링 
index :  13  name :  금융통화위원회 의사록(2021년도 제15차)(2021.7.29.)   파일 크롤링 
index :  14  name :  금융통화위원회 의사록(2021년도 제14차)(2021.7.15.)   파일 크롤링 
index :  15  name :  금융통화위원회 의사록(2021년도 제13차)(2021.6.30.)   파일

In [4]:
## generator Sentimenti
import os
from collections import defaultdict
from pandas import read_table
import numpy as np
import math
import datetime

from ekonlpy.sentiment import MPCK

mpck = MPCK()


class NaiveBayesClassifier:
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def load_corpusData(self, path):
        corpusData = read_table(path, sep=',', header=None, names=None, encoding='utf-8')
        corpusData = np.array(corpusData)

        return corpusData

    def count_words(self, training_set):
        counts = defaultdict(lambda: [0, 0])

        minutes_list = os.listdir('/content/text/') # 폴더 경로 수정

        file_list = minutes_list

       # print(training_set)

        for dataDate, label in training_set:
            dataDate_without_dot = dataDate.replace('.', '')

            for file in file_list:

                if len(file) <20 : # 해당 pdf 파일 제외한 파일들 입력넘어가기 위한 용도
                  #print("이상한 파일 체크 : ",len(file), " 길이 ", file)
                  continue
                #'content/text/' +  # 인코딩 euc-kr , cp949
                
                #str_datetime = str_to_date(file) # 숫자 날짜 전환용
                #print(file[24:],"  24 : -6  ",file[-6:])
                str_datetime = file[24:][:-6].replace('.','-').replace('(','').replace(')','')
                
                if str_datetime[-1]=='-' :
                  str_datetime = str_datetime[:-1]

                format = '%Y-%m-%d' # 2022-1-1 로 string 저장 처리를 위한 함수 
                #print(str_datetime)
                
                date_time_str = datetime.datetime.strptime(str_datetime, format)
                temp_str=date_time_str.isoformat()[:10]
                
                if dataDate == temp_str: # 해당 날짜처리후 날짜가 같은 것만 학습 ! 변동일 
                  
                  corpus = open('text/'+file, 'r', encoding='latin_1').read()
                  #corpus = open('text/'+file, 'r', encoding='utf-8').read()
                      
                  print("process minutes file name : ", file)

                  tokens = mpck.tokenize(corpus)
                  ngrams = mpck.ngramize(tokens)

                  for ngram in ngrams + tokens:
                      counts[ngram][0 if label == 1 else 1] += 1
            
        print(" 단어 갯수세기 부분 완료입니다. counts",counts)
        return counts

    def word_probabilities(self, counts, total_class0, total_class1, k):
        return [(w, (class0 + k) / (total_class0 + 2 * k), (class1 + k) / (total_class1 + 2 * k))
                for w, (class0, class1) in counts.items()]

    def train(self, trainfile_path):
        training_set = self.load_corpusData(trainfile_path)
#        print(training_set)
        positive = len([1 for _, label in training_set if label == 1])
        negative = len(training_set) - positive
        #print( poisi)
        word_counts = self.count_words(training_set)

        self.word_probs = self.word_probabilities(word_counts, positive, negative, self.k)

        for noOfWord in range(len(self.word_probs)):
            if self.word_probs[noOfWord][1] / self.word_probs[noOfWord][2] > 1:
                print("어디까지 실행이 되어냐요 prosi")
                with open('/content/positive.csv', 'a', encoding='utf-8') as f:
                    f.write(self.word_probs[noOfWord][0] + '\n')
            else:
                print("어디까지 실행이 되어냐요 nevi")
                with open('/content/negative.csv', 'a', encoding='utf-8') as f:
                    f.write(self.word_probs[noOfWord][0] + '\n')


def str_to_date(file) :
  file[24:][:-7].replace('(','').replace(')','').replace('.','-')

model = NaiveBayesClassifier() # 클래스 호줄

#model.train(trainfile_path='data/labeledCallRate.csv')


model.train(trainfile_path='labeledCallRate.csv')


process minutes file name :  금융통화위원회 의사록(2005년도 제12차)(2005.6.9) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제13차)(2005.6.23) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제14차)(2005.7.7) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제17차)(2005.8.11) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제19차)(2005.9.8) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제20차)(2005.9.22) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제21차)(2005.10.11) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제24차)(2005.11.10) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제26차)(2005.12.8) .pdf
process minutes file name :  금융통화위원회 의사록(2005년도 제27차)(2005.12.22) .pdf
process minutes file name :  금융통화위원회 의사록(2006년도 제1차)(2006.1.5) .pdf
process minutes file name :  금융통화위원회 의사록(2006년도 제2차)(2006.1.12) .pdf
process minutes file name :  금융통화위원회 의사록(2006년도 제3차)(2006.1.26) .pdf
process minutes file name :  금융통화위원회 의사록(2006년도 제4차)(2006.2.9) .pdf
process minutes file name 

In [ ]:
pwd

'/content'

In [ ]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint

In [ ]:
from konlpy.tag import Mecab
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# 사용자 사전 추가용
#https://somjang.tistory.com/entry/Google-Colab%EC%97%90%EC%84%9C-mecab-ko-dic-%EC%82%AC%EC%9A%A9%EC%9E%90-%EC%82%AC%EC%A0%84-%EC%B6%94%EA%B0%80%ED%95%98%EA%B8%B0
%cd Mecab-ko-for-Google-Colab/

!bash install_mecab-ko_on_colab190912.sh

%cd /content/

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-06-08 01:19:54--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://b

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
sentence = "안녕하세요 저는 상휴입니다."
temp_X = mecab.morphs(sentence)
temp_X

['안녕', '하', '세요', '저', '는', '상', '휴', '입니다', '.']

In [ ]:
# enonlp
! git clone https://github.com/entelecheia/eKoNLPy.git # 옆에 폴더에 설치 됩니다. 

! pip install /content/eKoNLPy

! pip install eKoNLPy --upgrade (for upgrade)

In [5]:
# measureToneScore
# -*- encoding: utf-8 -*-
import os
import pandas as pd
from collections import defaultdict
import csv
from ekonlpy.sentiment import MPCK

mpck = MPCK()

file_list = os.listdir('/content/text/')

# corpus = open('text/'+file, 'r', encoding='latin_1').read() 인코딩 참고 
positiveNgram = pd.read_csv('/content/positive.csv', sep='\n', header=None, names=['positiveNgram'], encoding='utf-8')
negativeNgram = pd.read_csv('/content/negative.csv', sep='\n', header=None, names=['negativeNgram'], encoding='utf-8')


for file in file_list:
    NoOfPositiveNgrams, NoOfnegativeNgrams = 0, 0

    minutes = open('/content/text/' + file, 'r', encoding='latin_1').read() # 인코딩 수정

    minutesTokens = mpck.tokenize(minutes)
    minutesNgrams = mpck.ngramize(minutesTokens)

    minutesNgramsTotal = minutesNgrams + minutesTokens

    print(file, '의 ngram 수: ', len(minutesNgramsTotal))
    print(file, '매칭 결과 : ')

    for mN in minutesNgramsTotal:
        for pN in positiveNgram.positiveNgram:
            if mN == pN:
                NoOfPositiveNgrams = NoOfPositiveNgrams + 1
                break
        #                 print('Positive match ngram: ')
        #                 print(mN, '==', pN)
        
        #for nN in negativeNgram.negativeNgram:
        #    if mN == nN:
        #        NoOfnegativeNgrams = NoOfnegativeNgrams + 1
        #        break
    #                 print('negative match ngram: ')
    #                 print(mN, '==', nN)


    # 감성사전과 매칭되는 ngram이 없는 경우
    if (NoOfPositiveNgrams + NoOfnegativeNgrams) == 0:
        print('매칭되는 ngram 없음 ')
        continue

    print('NoOfPositiveNgrams : ', NoOfPositiveNgrams)
    print('NoOfnegativeNgrams : ', NoOfnegativeNgrams)

    polarityScore_sentence = (NoOfPositiveNgrams - NoOfnegativeNgrams) / (NoOfPositiveNgrams + NoOfnegativeNgrams)

    print('어조지수 : ', polarityScore_sentence)
    print()

    with open('toneScore_newToneScoreGetcode_fromNewsBondDict.csv', 'a', encoding='utf-8') as f:
        f.write(file + "," + str(polarityScore_sentence) + '\n')

금융통화위원회 의사록(2015년도 제1차)(2015.1.15.) .pdf 의 ngram 수:  4
금융통화위원회 의사록(2015년도 제1차)(2015.1.15.) .pdf 매칭 결과 : 
NoOfPositiveNgrams :  3
NoOfnegativeNgrams :  0
어조지수 :  1.0

금융통화위원회 의사록(2014년도 제20차)(2014.10.23.) .pdf 의 ngram 수:  0
금융통화위원회 의사록(2014년도 제20차)(2014.10.23.) .pdf 매칭 결과 : 
매칭되는 ngram 없음 
금융통화위원회 의사록(2011년도 제30차)(2011.12.29) .pdf 의 ngram 수:  0
금융통화위원회 의사록(2011년도 제30차)(2011.12.29) .pdf 매칭 결과 : 
매칭되는 ngram 없음 
금융통화위원회 의사록(2007년도 제2차)(2007.1.11) .pdf 의 ngram 수:  2
금융통화위원회 의사록(2007년도 제2차)(2007.1.11) .pdf 매칭 결과 : 
NoOfPositiveNgrams :  2
NoOfnegativeNgrams :  0
어조지수 :  1.0

금융통화위원회 의사록(2007년도 제11차)(2007.5.10) .pdf 의 ngram 수:  0
금융통화위원회 의사록(2007년도 제11차)(2007.5.10) .pdf 매칭 결과 : 
매칭되는 ngram 없음 
금융통화위원회 의사록(2008년도 제18차)(2008.9.11) .pdf 의 ngram 수:  0
금융통화위원회 의사록(2008년도 제18차)(2008.9.11) .pdf 매칭 결과 : 
매칭되는 ngram 없음 
금융통화위원회 의사록(2010년도 제22차)(2010.11.16) .pdf 의 ngram 수:  0
금융통화위원회 의사록(2010년도 제22차)(2010.11.16) .pdf 매칭 결과 : 
매칭되는 ngram 없음 
금융통화위원회 의사록(2020년도 제22차)(2020.10.14)  .pdf 의 ngram 수:  0
금융통화위

In [1]:
## eKoNLP 설치용 

! pip install konlpy  # konlpy 설치 

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git  # mecab 설치용
# 사용자 사전 추가용
#https://somjang.tistory.com/entry/Google-Colab%EC%97%90%EC%84%9C-mecab-ko-dic-%EC%82%AC%EC%9A%A9%EC%9E%90-%EC%82%AC%EC%A0%84-%EC%B6%94%EA%B0%80%ED%95%98%EA%B8%B0
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh


%cd /content/
# enonlp
! git clone https://github.com/entelecheia/eKoNLPy.git # 옆에 폴더에 설치 됩니다. 

! pip install /content/eKoNLPy

! pip install eKoNLPy --upgrade (for upgrade)


## 설치된것 실행용 혹시 나 tagger 에러 난다면!! 런타임을 재실행 해볼것  
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab


from konlpy.tag import Mecab
mecab = Mecab()
sentence = "안녕하세요 저는 상휴입니다."
temp_X = mecab.morphs(sentence)
temp_X

print("eKoNLP 설치 완료!! 0.0")




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 7.0 MB/s 
     |████████████████████████████████| 453 kB 44.7 MB/s 
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 9.64 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-06-08 09:30:16--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%cd Mecab-ko-for-Google-Colab/
%cd /content/


/content


In [ ]:
pwd

'/content'

In [ ]:
! git clone https://github.com/entelecheia/eKoNLPy.git # 옆에 폴더에 설치 됩니다. 

! pip install /content/eKoNLPy

! pip install eKoNLPy --upgrade (for upgrade)
